In [5]:
import numpy as np
import sys
import os
from os.path import join as pjoin
from tqdm import tqdm
import glob

# root_rot_velocity (B, seq_len, 1)
# root_linear_velocity (B, seq_len, 2)
# root_y (B, seq_len, 1)
# ric_data (B, seq_len, (joint_num - 1)*3)
# rot_data (B, seq_len, (joint_num - 1)*6)
# local_velocity (B, seq_len, joint_num*3)
# foot contact (B, seq_len, 4)
def mean_variance(data_dir, save_dir, joints_num):
    file_list = glob.glob(os.path.join(data_dir, "**/*.npy"), recursive=True)
    data_list = []

    for file in tqdm(file_list):
        data = np.load(file)
        if np.isnan(data).any():
            print(file)
            continue
        data_list.append(data)

    data = np.concatenate(data_list, axis=0)
    print(data.shape)
    Mean = data.mean(axis=0)
    Std = data.std(axis=0)
    Std[0:1] = Std[0:1].mean() / 1.0
    Std[1:3] = Std[1:3].mean() / 1.0
    Std[3:4] = Std[3:4].mean() / 1.0
    Std[4: 4+(joints_num - 1) * 3] = Std[4: 4+(joints_num - 1) * 3].mean() / 1.0
    Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9] = Std[4+(joints_num - 1) * 3: 4+(joints_num - 1) * 9].mean() / 1.0
    Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3] = Std[4+(joints_num - 1) * 9: 4+(joints_num - 1) * 9 + joints_num*3].mean() / 1.0
    Std[4 + (joints_num - 1) * 9 + joints_num * 3: ] = Std[4 + (joints_num - 1) * 9 + joints_num * 3: ].mean() / 1.0

    assert 8 + (joints_num - 1) * 9 + joints_num * 3 == Std.shape[-1]

    np.save(pjoin(save_dir, 'Mean.npy'), Mean)
    np.save(pjoin(save_dir, 'Std.npy'), Std)

    return Mean, Std

In [2]:
# The given data is used to double check if you are on the right track.
reference1 = np.load('./HumanML3D/Mean.npy')
reference2 = np.load('./HumanML3D/Std.npy')

In [6]:
if __name__ == '__main__':
    data_dir = './HumanML3D/new_joint_comp_vecs/'
    save_dir = './HumanML3D/'
    mean, std = mean_variance(data_dir, save_dir, 22)
    print(mean)
    print(std)

 75%|███████▍  | 18696/24992 [07:29<02:21, 44.48it/s]

./HumanML3D/new_joint_comp_vecs/KIT/9/RightTurn10_poses.npy
./HumanML3D/new_joint_comp_vecs/KIT/9/M_RightTurn10_poses.npy


100%|██████████| 24992/24992 [10:03<00:00, 41.44it/s]


(6126232, 263)
[ 3.70707744e-06  9.69441135e-06  6.33491017e-03  7.64797449e-01
  5.69974594e-02  7.04173028e-01 -6.89538848e-03 -5.56907468e-02
  7.05058575e-01 -6.67006196e-03  6.39976588e-06  8.97828698e-01
 -2.79786270e-02  1.04032859e-01  4.01641220e-01  4.81610112e-02
 -1.04074851e-01  3.94424379e-01  4.98120077e-02  3.00718220e-05
  9.78556454e-01  8.99575464e-03  8.71708468e-02  8.99805352e-02
 -3.16759348e-02 -8.74276832e-02  8.59021619e-02 -2.96211541e-02
  3.01160326e-05  1.00806808e+00  2.99449563e-02  1.25657827e-01
  4.50403243e-02  5.95339164e-02 -1.25201494e-01  4.19692695e-02
  6.01923689e-02  3.02341796e-05  1.27807522e+00  3.29179317e-02
  6.94772825e-02  1.09927762e+00  2.46015545e-02 -6.92798793e-02
  1.09921718e+00  2.46867072e-02  5.37353735e-05  1.32972848e+00
  8.34050477e-02  1.49634957e-01  1.10154390e+00  2.55825408e-02
 -1.53001249e-01  1.10155416e+00  2.57966779e-02  2.12092638e-01
  9.72370386e-01  1.18709281e-02 -2.12828472e-01  9.76296365e-01
  1.270130

### Check if your data is correct. If it's aligned with the given reference, then it is right

In [4]:
abs(mean-reference1).sum()

0.0

In [5]:
abs(std-reference2).sum()

0.0